In [8]:
%run ./"0 Imports and functions.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
spectral_cube 0.6.1.dev22+g003ef16
spectral_cube file path /home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
reproject 0.8
3.3.4 Matplotlib
/home/ben/miniconda3/lib/python3.8/site-packages/matplotlib/__init__.py
1.23.1 Numpy
astrodendro_file: /home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
Results will be saved to Directory ./Result Files
Cubes will be saved and loaded with Directory ./Spectral Cubes
Plots will be saved to Directory ./Plots
Dendrograms will be saved to Directory ./Dendrograms


In [9]:
#extract the h13CO line from SiO
#File ="H13COp_J1-0_SiO_J2-1.cube.fits"
#File = "member.uid___A001_X129e_Xb6.NGC_253_sci.spw25.cube.I.pbcor.fits"
File = "H13CO_Plus_1_0.fits"
restfreq = 86.7542884 *10**9 *u.Hz
scB = (SpectralCube.read("Spectral Cubes/"+File) ).with_spectral_unit(u.km/u.s,velocity_convention="radio",rest_value=restfreq).spectral_slab(- 120*u.km/u.s + 250*u.km/u.s, 120*u.km/u.s+ 250*u.km/u.s)  # Crop out velocities we don't care about   # Change units from Hz to km/s in case they are in Hz


#scB.write("Spectral Cubes/H13COp_J1-0_SiO_J2-1.cube_Isolated.fits",overwrite=True)
scB.write("Spectral Cubes/H13CO_Plus_J_1_0_Isolated.fits")


In [10]:
# Pointing_Information

Pointing_Information = {}

#This is the stuff that needs changing between cubes

Pointing_Information["Original_File_Name"] = "H13CO_Plus_J_1_0_Isolated.fits" #the name of the initial SC file.
File_Descriptor="NGC_253_H13CO+_J1-0_"
Pointing_Information["File_Descriptor"] = File_Descriptor
Pointing_Information["target"] = "NGC253"#or the cmz
Pointing_Information["center"] = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs') #the center of NGC253
Pointing_Information["crop_center"] = SkyCoord('00h47m33.14s' ,'-25d17m17.52s',frame='icrs') #the center of NGC253, crop around this point

desired_beam_size = 4.8*u.pc #I add this to the PI later. choose this based on the largest common beam size between the compared observations
distance = 3.5*u.Mpc
Pointing_Information["distance"] = distance.to(u.Mpc) #in Mpc
Pointing_Information["target_image_rotation"]=33*u.deg #this is the rotation of the specific image, not the target. (use clockwise rotation angle)
Pointing_Information["target_inclination"]=78*u.deg
Pointing_Information["target_velocity"]=250*u.km/u.s #the speed NGC253 is moving away from us
Pointing_Information["vaxis"]=0 #which axis is the velocity
Pointing_Information["desired_velocity_resolution"] = 3.4*u.km/u.s
ovs = 5 #how much do you desire to oversample the beam by
desired_FOV = [70,360]*u.pc# pc normal to the disk, across the disk
Pointing_Information["Corresponding_Continuum"] = 'Continuum_Reproject.fits' # this is for the band 7, not for HCN J1-0 #The continuum image for this band
#                                                 '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits' # this is the JCMT continuum for 850microns (the CO 3-2 and HCN, HCO 4-3)


print("done")

gc.collect

done


<function gc.collect(generation=2)>

In [11]:

#This stuff will input automagically if the rest is correct

sc = SpectralCube.read("Spectral Cubes/"+Pointing_Information["Original_File_Name"])
header = sc.header
print()
try:
    freq = header["RESTFREQ"]*u.Hz# assuming the header is in Hz
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFREQ"]#
    Pointing_Information['restfreq']=freq         
except:
    freq = header["RESTFRQ"]*u.Hz#
    Pointing_Information['wavelength']=299792458*u.m/header["RESTFRQ"]#            
    Pointing_Information['restfreq']=freq 

######calculate teh beam size of the original image:

if (header['CUNIT1'].find("deg")!=-1):
    CUNIT = 1*u.degree
    Pointing_Information["CUNIT"]=CUNIT
else:
    print("The header should show CUNIT in degrees. If not, just fix this or write CUNIT = the unit it says in the header")
    del jhgasdhgjkahsdkgdfjhsgjgjsdhkfjghjd #this causes an error and stops execution 
    
beam_major =  (header["BMAJ"]*CUNIT).to(u.arcsec) #degrees beam size -> arcsec
beam_minor =  (header["BMIN"]*CUNIT).to(u.arcsec)


Pointing_Information["original_BMAJ"]=beam_major
Pointing_Information["original_BMIN"]=beam_minor
Pointing_Information["original_BMAJ_pc"]=beam_major.to(u.rad)*Pointing_Information['distance']
Pointing_Information["original_BMIN_pc"]=beam_minor.to(u.rad)*Pointing_Information['distance']
Pointing_Information["desired_beam_size"] = desired_beam_size #ill put it in a circular beam at this size


#this accounts for elliptical beams:            
Pointing_Information['original_pixel_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_pixel_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix
Pointing_Information['original_spatial_scale_x'] = (abs(header["CDELT1"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']
Pointing_Information['original_spatial_scale_y'] = (abs(header["CDELT2"])*CUNIT.to(u.rad))/u.pix*Pointing_Information['distance']#convert to pc using the distance

average_pixel=np.sqrt((abs(header["CDELT1"])*CUNIT.to(u.arcsec))/u.pix*(abs(header["CDELT2"])*CUNIT.to(u.arcsec))/u.pix)

Pointing_Information['original_beam_oversampling_MAJ'] = beam_major/average_pixel
Pointing_Information['original_beam_oversampling_MIN'] = beam_minor/average_pixel
Pointing_Information['desired_beam_oversampling'] = ovs
#Pointing_Information['orig_FOV']= Crop_Around_Center(sc,Pointing_Information['target_image_rotation'],Pointing_Information['center'],desired_FOV,Pointing_Information['distance'])[1] #returns the current fov
Pointing_Information['desired_FOV']=desired_FOV

######

Cube_Information = Update_Cube_Information(Pointing_Information,Pointing_Information["Original_File_Name"])


# 1. Reprojection

In [12]:
gc.collect()

#
# def Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg]):

Reproject_To_Region(Pointing_Information,Cube_Information,i_step=30,Cube_Name_Save='',Force_Origin=[False,[0,0]*u.deg,[0,0]*u.deg])
    
# This is the cube name that they are being saved to unless you changed it inside the reproject functions definiton

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

gc.collect()

print("done")


begin step: 0 of 3
starting cube rotation
finished cube rotation
start beam convolution
0.0 1.494114997421817
2.8440490809084506e-09 3.037156523715571e-11
convolve end

fov crop start 

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 471 453 Pixel FOV: [66, 342]
cropping cube. rotation: 0.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
Cropped to  [ 70. 360.] pc
check max SC value after crop: 0.0036201674795637464 SC shape: (30, 130, 682)
fov cropped

start reprojection

check max SC value: 0.0036201674795637464 SC shape: (30, 130, 682)

end reprojection

check max SC value: 0.0035719313341757053 SC shape: (30, 624, 1036)
fov crop start 2 

Center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> Pixel center: 482 537 Pixel FOV: [36, 187]
cropping cube. rotation: 0.0 deg center: <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)> crop to: [ 70. 360.] pc
Cropped to  [ 7

# 1a. Splice the reprojected cubes back together

In [13]:
# First, splice the partwise cubes back together

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Splice_vels(Cube_Name_Load=Cube_Name_Save)

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

gc.collect()

print("done")

Loaded 0_4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 1_4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected.fits
Loaded 2_4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected.fits
[Errno 2] No such file or directory: 'Spectral Cubes/3_4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected.fits'
------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_473733/1580511263.py", line 11, in Splice_vels
    sc=SpectralCube.read(("Spectral Cubes/"+Cube_Name_Load_p))
  File "/home/ben/.local/lib/python3.8/site-packages/spectral_cube/io/core.py", line 118, in __call__
    return registry.read(BaseSpectralCube, filename, *args, **kwargs)
  File "/home/ben/.local/lib/python3.8/site-packages/astropy/io/registry/compat.py", line 43, in wrapper
    return getattr(registry, method_name)(*args, **kwargs)
  File "/home/ben/.local/lib/python3.8/site-packages/astropy/io/registry/core.py", line 184, in read
    fil

# 1b. Velocity reprojection

In [14]:
Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Load the spliced cube

sc = SpectralCube.read(("Spectral Cubes/"+Cube_Name_Save)) 

# Reproject the spliced Cube's veloctiy axis

Repo_Velocity(sc,Cube_Name_Save,Cube_Information)

# Here is the new cube's name

Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

del sc

gc.collect()

Start velocity reprojection of 4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected.fits
Smoothed to Gaussian Kernel of width 0.17473778472245002
Wrote reprojected cube to Spectral Cubes/4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected_3.4_vel_res_.fits


0

# 1c. Fix repeated pixels

In [15]:
# after the reprojection, there may be duplicated data from the fact that it just fills in stuff 
# expecting new data to be there. This fixes it by checking to see if it matches other data, then removes
# the repeated slices. in other words, it may be set to reproject to 0-500 km/s through the velocity channels
# but there may only be data between 100-400, so it will fill it in with copied data channels. This will check for those.



# Update to the current cube name

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

# Run the function to crop the nan values out of the cube

Remove_Repeated_Pixels(Cube_Name_Save)

Good data starting from channel 38 ; start has been cropped to that channel
Good data ending at channel 109 ; end has been cropped to that channel
Cropped cube saved as  Spectral Cubes/Cropped_4.8pc_beam_NGC_253_H13CO+_J1-0_70.0x360.0pc_reprojected_3.4_vel_res_.fits


# x. save cube information

In [16]:
# Now that all the data reduction is concluded, save information about the new cubes

Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)

{'Original_File_Name': 'H13CO_Plus_J_1_0_Isolated.fits', 'File_Descriptor': 'NGC_253_H13CO+_J1-0_', 'target': 'NGC253', 'center': <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)>, 'crop_center': <SkyCoord (ICRS): (ra, dec) in deg
    (11.88808333, -25.2882)>, 'distance': <Quantity 3.5 Mpc>, 'target_image_rotation': <Quantity 0. deg>, 'target_inclination': <Quantity 78. deg>, 'target_velocity': <Quantity 250. km / s>, 'vaxis': 0, 'desired_velocity_resolution': <Quantity 3.4 km / s>, 'Corresponding_Continuum': 'Continuum_Reproject.fits', 'wavelength': <Quantity 0.00345565 m>, 'restfreq': <Quantity 8.67542884e+10 Hz>, 'CUNIT': <Quantity 1. deg>, 'original_BMAJ': <Quantity 0.28256428 arcsec>, 'original_BMIN': <Quantity 0.20722991 arcsec>, 'original_BMAJ_pc': <Quantity 4.79468605e-06 Mpc rad>, 'original_BMIN_pc': <Quantity 3.51637638e-06 Mpc rad>, 'desired_beam_size': <Quantity 4.8 pc>, 'original_pixel_scale_x': <Quantity 0.031 arcsec / pix>, 'original_pixel_scale_y': <Quanti

# xb. Noise matching 


if Pointing_Information["noise_matched"]:
    print("The cube already has spatially uniform noise")
    pass
else
    # Return an input cube matched to the given nosie

    Cube_Name_Save = str(Pointing_Information["desired_beam_size"].value)+"pc_beam_"+Pointing_Information["File_Descriptor"]+str(Pointing_Information["desired_FOV"][0].value)+"x"+str(Pointing_Information["desired_FOV"][1].value)+'pc_'+'reprojected.fits'
    Cube_Name_Save = "Cropped_"+Cube_Name_Save[0:len(Cube_Name_Save)-5]+"_"+str(Pointing_Information["desired_velocity_resolution"].value)+'_vel_res_'+'.fits'

    Q  = SpectralCube.read("Spectral Cubes/"+Cube_Name_Save)

    NM_Cube = Noise_matching(Input_Cube=Q,m=(.115*u.K))


    Cube_Name_Save = "NM_"+Cube_Name_Save

    NM_Cube.write("Spectral Cubes/"+Cube_Name_Save)

    Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

    import pickle 

    with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
        pickle.dump(Cube_Information, file)

    with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
        pickle.dump(Pointing_Information, file)

    print(Cube_Information)


Cube_Information = Update_Cube_Information(Pointing_Information,Cube_Name_Save)

import pickle 

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Cube_Information", 'wb') as file:
    pickle.dump(Cube_Information, file)

with open("Spectral Cubes/Cube Information/"+File_Descriptor+"_Pointing_Information", 'wb') as file:
    pickle.dump(Pointing_Information, file)

print(Cube_Information)